Decision trees involve the greedy selection of the best split point from the dataset at each step. This algorithim makes decision trees susceptible to high variance if they are not pruned. One way around this is to  create multiple trees with different samples of the training dataset and combining predictions, this is called bootsrap aggregation or bagging. However if trees have similar split points then it mitigates the variance that was sought and predictions are likely to be similar. 

 
Using a decision tree to find the optimal decision is called solving the tree. To sovle a decision tree we work backwards. 

Descision trees are easy to read and use, this is very useful in practice. 

decision trees are fragile to changes in data value and lack reproducability- for that reason they are not as useful for prediction but because they are easy to interpret that can be a great tool for data exploration


random forest are built to immediately address the concerns of reproducability and sensitivity to changes in data

decision trees look for a split on every variable on every node while random forests look for a split on one variable- the variable with the largest association with target among candidate explanatory varianles, whih is a subset of the total list of explanatory variables. 

Each tree is grown by a subset of the explanatory variables at each node and a random subset of the sample for each tree in the forest. 

There is the bagged data, the subset chosen to generate the tree, and the ou-of-bag data- that is available to test the accuracy of the trees. 

The trees themselves are not interpretted, they are used collectively to rank the importance of variables in predicting the target of interest.

Split points are determimined by either the sum squared error for regression problemsor the gini cost funciton for claddification problems - 

splitting continues until the nodes contain the minimum number of training examples or a maximum depth is reached

This is an exhaustive and greedy algorithm.

make sure to plot split points and gini imputity or something 


In [2]:
import pandas as pd 
import seaborn
from IPython.display import display

headers = ['index','refractive index', 'sodium', 'magnesium', 'aluminum', 'silicon', 'potassium', 'calcium', 'barium', 'iron', 'type of glass']
df = pd.read_csv('glass_data.csv', names = headers)

In [1]:
# display(df.head(5))
# display(df.describe()) 

In [31]:
def split_data(column, value, dataset):
    split_point = df[column] > value 
    right = df[split_point]
    left = df[~split_point]
    return [right, left]

def gini_impurity(groups):
    n_samples = sum([len(group) for group in groups]) #add float 
    gini = 0.0
    for group in groups:
        group_size = float(len(group))
        if group_size == 0:
            continue
        score = sum(group.iloc[:, -1].value_counts().map(lambda p: (p/group_size)* (p/group_size)))
        gini += (1.0 - score) * (group_size / n_samples)
    return gini


def get_split(dataset):
    s_idx, s_value, s_score, s_groups = 999, 999, 999, None
    for row in dataset.values:
        for idx, value in enumerate(row):
            groups = split_data(dataset.columns[idx], value, dataset)
            gini = gini_impurity(groups)
            if gini < s_score:
                s_idx, s_value, s_score, s_groups = dataset.columns[idx], value, gini, groups
    return {'column':s_idx, 'value':s_value, 'groups':s_groups}



In [73]:
sample = df.sample(10).drop('index', axis=1)#drop the final column too
s = get_split()

#minimum nodes
#maximum tree depth
#deal with a split that returns all nodes on one side



{'column': 'type of glass',
 'groups': [     index  refractive index  sodium  magnesium  aluminum  silicon  potassium  \
  70      71           1.51574   14.86       3.67      1.74    71.87       0.16   
  71      72           1.51848   13.64       3.87      1.27    71.96       0.54   
  72      73           1.51593   13.09       3.59      1.52    73.10       0.67   
  73      74           1.51631   13.34       3.57      1.57    72.87       0.61   
  74      75           1.51596   13.02       3.56      1.54    73.11       0.72   
  75      76           1.51590   13.02       3.58      1.51    73.12       0.69   
  76      77           1.51645   13.44       3.61      1.54    72.39       0.66   
  77      78           1.51627   13.00       3.58      1.54    72.83       0.61   
  78      79           1.51613   13.92       3.52      1.25    72.88       0.37   
  79      80           1.51590   12.82       3.52      1.90    72.86       0.69   
  80      81           1.51592   12.86       3.52

In [ ]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [ ]:

from graphviz import Digraph

dot = Digraph(comment='The Round Table')

dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

dot